In [3]:
!pip freeze | grep requests

requests==2.32.3
requests-toolbelt==1.0.0


In [4]:
BASE = "http://localhost:5055"

SOFTWARE_LIST= "/api/softwares"
SOFTWARE_VERSIONS = "/get_versions"
SOFTWARE_DEPENDENCIES="/api/dependencies" #Params: name, version
SOFTWARE_SBOM="/api/sbom" #POST: name, version
SOFTWARE_VULNERABILITES="/api/vulnerabilities"


In [5]:
import requests as r

name = "aws-sdk-cpp"
version = "1.11.117"

list = r.get(BASE+SOFTWARE_LIST)
print(list.text)
versions=r.get(f"{BASE}{SOFTWARE_VERSIONS}?name={name}")
print(versions.text)
dependencies=r.get(f"{BASE}{SOFTWARE_DEPENDENCIES}?name={name}&version={version}")
print(dependencies.text)
vulns = r.get(f"{BASE}{SOFTWARE_VULNERABILITES}?name={name}&version={version}")
print(vulns.text)
sbom = r.post(BASE+SOFTWARE_SBOM, json= {
    "name": name, "version": version
})
print(sbom.text)

[]

"{\"software\": \"aws-sdk-cpp\", \"version\": \"1.11.117\", \"dependencies\": [\"https://w3id.org/secure-chain/SoftwareVersion/threads#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/zlib#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/openssl#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/project#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/testing-resources#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/git#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/c#2\", \"https://w3id.org/secure-chain/SoftwareVersion/crypto#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/aws#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/android#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/aws-cpp-sdk-petstore#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/%24%7Btarget%7D#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/cmake_android_ndk_toolchain#%2A\", \"https://w3id.org/secure-chain/SoftwareVersion/pythoninterp#%2A\",

In [ ]:
list = r.get(BASE+SOFTWARE_LIST)

versions=r.get(f"{BASE}{SOFTWARE_VERSIONS}?name={name}")

dependencies=r.get(f"{BASE}{SOFTWARE_DEPENDENCIES}?name={name}&version={version}")

vulns = r.get(f"{BASE}{SOFTWARE_VULNERABILITES}?name={name}&version={version}")

sbom = r.post(BASE+SOFTWARE_SBOM, json= {
    "name": name, "version": version
})


In [19]:
%%capture
%pip install google-generativeai
%pip install -U instructor

In [24]:
# Setup lib
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI
import google.generativeai as genai
from typing import Literal

from dotenv import load_dotenv
load_dotenv()

# Define User intent enum
class ClassificationResponse(BaseModel):
    """
    A few-shot example of text classification:

    Examples:
    - "Get me the list of software!": SOFTWARE_LIST
    - "What are the versions for this software": SOFTWARE_VERSIONS
    - "What are libraries are used for aws-sdk": SOFTWARE_DEPENDENCIES
    - "List the vulnerabilities in linux?": SOFTWARE_VULNERABILITES
    - "Generate the SBOM for whisper": SOFTWARE_SBOM
    - "Write me a haiku": UNKNOWN
    """

    chain_of_thought: str = Field(
        ...,
        description="The chain of thought that led to the prediction.",
    )
    label: Literal["SOFTWARE_LIST", "SOFTWARE_VERSIONS", "SOFTWARE_DEPENDENCIES",
                    "SOFTWARE_VULNERABILITES", "SOFTWARE_SBOM", "UNKNOWN"] = Field(
        ...,
        description="The predicted class label.",
    )

# Define output
class Software(BaseModel):
    name: str
    version: str
    intent: ClassificationResponse

ollama_setup = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
gemini_setup = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/", api_key=os.getenv("GOOGLE_API_KEY"))
client = instructor.from_openai(
    gemini_setup,
    mode=instructor.Mode.JSON,
)


query = "What are the vulnerabilities in aws-sdk-cpp?"
resp = client.chat.completions.create(
    # model="llama3.2",
    model="gemini-1.5-flash",
    n=1,
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    response_model=Software,
)
print(resp.model_dump_json(indent=2))


{
  "name": "aws-sdk-cpp",
  "version": "unspecified",
  "intent": {
    "chain_of_thought": "The question asks about vulnerabilities in a specific software, aws-sdk-cpp.  This falls under the category of SOFTWARE_VULNERABILITES.",
    "label": "SOFTWARE_VULNERABILITES"
  }
}


In [27]:
queries = [
    "What versions of openai exists?",
    "What are the known vulnerabilities in version 2.2.20 of aws-sdk-cpp?",
    "Generate an SBOM for instructor v1.1.100"
]

def user_intent(query):
    resp = client.chat.completions.create(
    # model="llama3.2",
    model="gemini-1.5-flash",
    n=1,
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    response_model=Software,
    )

    return resp

for query in queries:
    intent = user_intent(query)
    print(intent)

name='openai' version='unknown' intent=ClassificationResponse(chain_of_thought='The question asks for versions of openai.  OpenAI does not have publicly listed version numbers in the same way that software packages do.  Their models and APIs are updated continuously.', label='UNKNOWN')
name='aws-sdk-cpp' version='2.2.20' intent=ClassificationResponse(chain_of_thought='The user is asking for known vulnerabilities in a specific version of the aws-sdk-cpp. This falls under the category of SOFTWARE_VULNERABILITES.', label='SOFTWARE_VULNERABILITES')
name='instructor' version='v1.1.100' intent=ClassificationResponse(chain_of_thought='The user is requesting an SBOM, which is a Software Bill of Materials.  This is directly related to software composition analysis and identifying the components of a piece of software. Therefore, the intent is to generate an SBOM.', label='SOFTWARE_SBOM')
